In [1]:
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0) 

### Test the loaded models and tokenizers.

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')
train_path = '../dataset/UCF/train/'
test_path = '../dataset/UCF/test/'

#### Load UCF files

In [3]:
broad_cat_dict={'I': ['A'],
                'II': ['B'],
                'III': ['C', 'D'],
                'IV': ['E', 'F', 'G', 'H'],
                'V': ['I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'],
                'VI': ['Q'],
                'VII': ['R', 'S', 'T', 'U', 'V', 'W'],
                'VIII': ['X'],
                'IX': ['Y'],
                'X': ['Z'],
              }

In [4]:
import pandas as pd
import os

def split_dataset(data_path):
  file_list=os.listdir(data_path)
  df_data=pd.DataFrame()
  for file in file_list:
    print(data_path+file)
    df_data=pd.concat([df_data, pd.read_pickle(data_path+file, compression='gzip')])

  df_data['mission_prgrm_spellchk']=df_data['TAXPAYER_NAME']+' '+df_data['mission_spellchk']+' '+df_data['prgrm_dsc_spellchk'] # Using spell-checked.
  df_data['broad_cat']=df_data['NTEE1'].apply(ntee2cat)
  print(len(df_data['mission_prgrm_spellchk']), len(df_data['NTEE1'].drop_duplicates()), len(df_data['broad_cat'].drop_duplicates()))
  return df_data

def ntee2cat(string):
  return [s for s in broad_cat_dict.keys() if string in broad_cat_dict[s]][0]


In [5]:
df_train = split_dataset(train_path)

../dataset/UCF/train/df_ucf_train.pkl.gz_4of4
../dataset/UCF/train/df_ucf_train.pkl.gz_0of4
../dataset/UCF/train/df_ucf_train.pkl.gz_3of4
../dataset/UCF/train/df_ucf_train.pkl.gz_2of4
../dataset/UCF/train/df_ucf_train.pkl.gz_1of4
154424 25 9


In [6]:
df_test = split_dataset(test_path)

../dataset/UCF/test/df_ucf_test.pkl.gz
38607 25 9


In [8]:
!pip install transformers

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [11]:
from sklearn import preprocessing

#encoding function
def padding_text(dataset):
  input_ids=[]
  attention_masks=[]
  text_list =  dataset['mission_prgrm_spellchk']
  for text in text_list:
    encode_plus = tokenizer.encode_plus(            
                        text,    
                        add_special_tokens = True,
                        max_length = 512,        
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt')   
    input_ids.append(encode_plus['input_ids'])
    attention_masks.append(encode_plus['attention_mask'])
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  lb_broad_cat = preprocessing.LabelEncoder().fit_transform(dataset['broad_cat'])
  lb_major_group = preprocessing.LabelEncoder().fit_transform(dataset['NTEE1'])
  lb_broad_cat = torch.tensor(lb_broad_cat)
  lb_major_group = torch.tensor(lb_major_group)
  return input_ids, attention_masks, lb_broad_cat, lb_major_group

In [12]:
#encoding train and test dataset
input_ids_train, attention_masks_train, lb_broad_group_train, lb_major_group_train = padding_text(df_train)
input_ids_test, attention_masks_test, lb_broad_cat_test, lb_major_group_test = padding_text(df_test)

In [13]:
from torch.utils.data import TensorDataset
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import *

#combining data and loading Bert
def train_test_dataset(train_label, test_label, num_label, batch_size):
  
  train_dataset = TensorDataset(input_ids_train, attention_masks_train, train_label)
  test_dataset = TensorDataset(input_ids_test, attention_masks_test, test_label)
  

  train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset),
              batch_size = batch_size )

  validation_dataloader = DataLoader(test_dataset,
              sampler = SequentialSampler(test_dataset), 
              batch_size = batch_size)
  
  global model

  model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                                num_labels=num_label) 
  model.cuda()
  
  
  return train_dataloader, validation_dataloader

In [14]:
from transformers import get_linear_schedule_with_warmup

#function for adjusting parameters
def parameters(learning_rate, eps, num_epoch):
  
  global epochs
  epochs = num_epoch
  optimizer = AdamW(model.parameters(),
                    lr = learning_rate, 
                    eps = eps 
                  )
  
  total_steps = len(train_dataloader) * epochs

  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, 
                                              num_training_steps = total_steps)
  return optimizer, scheduler

In [15]:
import numpy as np
#function that will evaluate the accuracy of the model
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [16]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [17]:
import random

#loading data into the Bert model

def run_bert(train_dataloader, validation_dataloader, optimizer, scheduler):

  seed = 42
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) 

  loss_values = []

  for epoch_i in range(0, epochs):

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      t0 = time.time()

      total_loss = 0

      
      model.train()
      model.resize_token_embeddings(len(tokenizer))

    
      for step, batch in enumerate(train_dataloader):

          if step % 2000 == 0 and not step == 0:
              elapsed = format_time(time.time() - t0)
              
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          
          model.zero_grad() 
          
          outputs = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask, 
                      labels=b_labels)
          
          loss = outputs[0] 
          total_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          
          optimizer.step()

          scheduler.step()

      avg_train_loss = total_loss / len(train_dataloader)            
      
      loss_values.append(avg_train_loss)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
          
      #               Validation

      print("\n")
      print("Validation...")

      t0 = time.time()

      model.eval()

      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      for batch in validation_dataloader:
          
          batch = tuple(t.to(device) for t in batch)
          
          b_input_ids, b_input_mask, b_labels = batch

          with torch.no_grad():        

      
              outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask)
          logits = outputs[0]

          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          
          eval_accuracy += tmp_eval_accuracy

          nb_eval_steps += 1

      # Report the final accuracy for this validation run.
      print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
      print("  Validation took: {:}".format(format_time(time.time() - t0)))

  print("")
  print("Training complete!")

In [18]:
#load broad group label
train_dataloader, validation_dataloader= train_test_dataset(train_label = lb_major_group_train, 
                                                            test_label = lb_major_group_test, 
                                                            num_label = 25, batch_size = 8)

In [19]:
optimizer, scheduler = parameters(learning_rate = 5e-5, eps = 1e-12, num_epoch = 4)
run_bert(train_dataloader, validation_dataloader, optimizer, scheduler)


======== Epoch 1 / 4 ========
Training...
  Batch 2,000  of  19,303.    Elapsed: 0:09:38.
  Batch 4,000  of  19,303.    Elapsed: 0:19:12.
  Batch 6,000  of  19,303.    Elapsed: 0:28:47.
  Batch 8,000  of  19,303.    Elapsed: 0:38:23.
  Batch 10,000  of  19,303.    Elapsed: 0:47:52.
  Batch 12,000  of  19,303.    Elapsed: 0:57:22.
  Batch 14,000  of  19,303.    Elapsed: 1:06:52.
  Batch 16,000  of  19,303.    Elapsed: 1:16:23.
  Batch 18,000  of  19,303.    Elapsed: 1:25:53.

  Average training loss: 0.67
  Training epcoh took: 1:32:05


Validation...
  Accuracy: 0.86
  Validation took: 0:07:23

======== Epoch 2 / 4 ========
Training...
  Batch 2,000  of  19,303.    Elapsed: 0:09:36.
  Batch 4,000  of  19,303.    Elapsed: 0:19:07.
  Batch 6,000  of  19,303.    Elapsed: 0:28:38.
  Batch 8,000  of  19,303.    Elapsed: 0:38:10.
  Batch 10,000  of  19,303.    Elapsed: 0:47:48.
  Batch 12,000  of  19,303.    Elapsed: 0:57:28.
  Batch 14,000  of  19,303.    Elapsed: 1:07:02.
  Batch 16,000  

In [20]:
#load broad group label
train_dataloader, validation_dataloader= train_test_dataset(train_label = lb_broad_group_train, test_label = lb_broad_cat_test, num_label = 9, batch_size = 8)

In [21]:
optimizer, scheduler = parameters(learning_rate = 5e-5, eps = 1e-12, num_epoch = 4)
run_bert(train_dataloader, validation_dataloader, optimizer, scheduler)


======== Epoch 1 / 4 ========
Training...
  Batch 2,000  of  19,303.    Elapsed: 0:09:30.
  Batch 4,000  of  19,303.    Elapsed: 0:19:13.
  Batch 6,000  of  19,303.    Elapsed: 0:28:43.
  Batch 8,000  of  19,303.    Elapsed: 0:38:20.
  Batch 10,000  of  19,303.    Elapsed: 0:47:57.
  Batch 12,000  of  19,303.    Elapsed: 0:57:35.
  Batch 14,000  of  19,303.    Elapsed: 1:07:08.
  Batch 16,000  of  19,303.    Elapsed: 1:16:44.
  Batch 18,000  of  19,303.    Elapsed: 1:26:25.

  Average training loss: 0.50
  Training epcoh took: 1:32:44


Validation...
  Accuracy: 0.89
  Validation took: 0:07:23

======== Epoch 2 / 4 ========
Training...
  Batch 2,000  of  19,303.    Elapsed: 0:09:30.
  Batch 4,000  of  19,303.    Elapsed: 0:18:59.
  Batch 6,000  of  19,303.    Elapsed: 0:28:32.
  Batch 8,000  of  19,303.    Elapsed: 0:38:12.
  Batch 10,000  of  19,303.    Elapsed: 0:47:44.
  Batch 12,000  of  19,303.    Elapsed: 0:57:17.
  Batch 14,000  of  19,303.    Elapsed: 1:06:47.
  Batch 16,000  